# Credit Risk Resampling Techniques

In [61]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [64]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/lending_data.csv'

# Split the Data into Training and Testing

In [65]:
y = df["loan_status"]
#y = pd.get_dummies(y)
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)

# Create our features


# Create our target


In [66]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [67]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [68]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [69]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaly = StandardScaler()

In [70]:
# Fit the Standard Scaler with the training data
scaly.fit(X)

StandardScaler()

In [71]:
# Scale the training and testing data
scalier = scaly.transform(X)
scalier[:5]

array([[ 0.42730427,  0.4268375 ,  0.42740435,  0.66857141,  0.61614258,
         1.04399575,  0.42740435, -0.994957  ,  1.22752672, -0.33999088],
       [-0.67148676, -0.67491817, -0.67155173, -0.80216566, -0.43404935,
        -0.67397306, -0.67155173, -0.994957  ,  1.22752672, -0.33999088],
       [-0.38484562, -0.370249  , -0.37292236, -0.34442423, -0.43404935,
        -0.67397306, -0.37292236, -0.994957  , -0.81464622,  2.9412554 ],
       [ 0.42730427,  0.41784357,  0.41545918,  0.65534557,  0.61614258,
         1.04399575,  0.41545918, -0.994957  ,  1.22752672, -0.33999088],
       [ 0.47507779,  0.45606775,  0.4512947 ,  0.69487335,  0.61614258,
         1.04399575,  0.4512947 ,  1.00506856, -0.81464622, -0.33999088]])

# Simple Logistic Regression

In [72]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [73]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [74]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [75]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [76]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

nrs = RandomOverSampler(random_state=1)
X_resampled, y_resampled = nrs.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [77]:
# Train the Logistic Regression model using the resampled data
NRSmodel = LogisticRegression(solver='lbfgs', random_state=1)
NRSmodel.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [78]:
# Display the confusion matrix
NRS_y_pred = model.predict(X_resampled)
confusion_matrix(y_resampled, NRS_y_pred)

array([[50679,  5598],
       [  287, 55990]])

In [79]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_resampled, NRS_y_pred)

0.9477139861755246

In [80]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled, NRS_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.99      0.90      0.99      0.95      0.95      0.89     56277
   low_risk       0.91      0.99      0.90      0.95      0.95      0.90     56277

avg / total       0.95      0.95      0.95      0.95      0.95      0.90    112554



### SMOTE Oversampling

In [81]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

SMOTE_X_resampled, SMOTE_y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
Counter(SMOTE_y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [82]:
# Train the Logistic Regression model using the resampled data
SMOTE_model = LogisticRegression(solver='lbfgs', random_state=1)
SMOTE_model.fit(SMOTE_X_resampled, SMOTE_y_resampled)

LogisticRegression(random_state=1)

In [83]:
# Calculated the balanced accuracy score
SMOTE_y_pred = SMOTE_model.predict(SMOTE_X_resampled)
balanced_accuracy_score(SMOTE_y_resampled, SMOTE_y_pred)

0.9946159176928409

In [84]:
# Display the confusion matrix
confusion_matrix(SMOTE_y_resampled, SMOTE_y_pred)

array([[55989,   288],
       [  318, 55959]])

In [85]:
# Print the imbalanced classification report
print(classification_report_imbalanced(SMOTE_y_resampled, SMOTE_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.99      0.99      0.99      0.99      0.99      0.99     56277
   low_risk       0.99      0.99      0.99      0.99      0.99      0.99     56277

avg / total       0.99      0.99      0.99      0.99      0.99      0.99    112554



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [86]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
US_X_resampled, US_y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(US_y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [48]:
# Train the Logistic Regression model using the resampled data
US_model = LogisticRegression(solver='lbfgs', random_state=1)
US_model.fit(US_X_resampled, US_y_resampled)

LogisticRegression(random_state=1)

In [58]:
# Display the confusion matrix
US_y_pred = US_model.predict(US_X_resampled)
confusion_matrix(US_y_resampled, US_y_pred)

array([[1828,   47],
       [ 232, 1643]])

In [59]:
# Calculate the balanced accuracy score
balanced_accuracy_score(US_y_resampled, US_y_pred)

0.9256

In [60]:
# Print the imbalanced classification report
print(classification_report_imbalanced(US_y_resampled, US_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.89      0.97      0.88      0.93      0.92      0.86      1875
   low_risk       0.97      0.88      0.97      0.92      0.92      0.85      1875

avg / total       0.93      0.93      0.93      0.93      0.92      0.85      3750



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [54]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
SM_X_resampled, SM_y_resampled = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(SM_y_resampled)

Counter({'high_risk': 55513, 'low_risk': 55928})

In [55]:
# Train the Logistic Regression model using the resampled data
SM_model = LogisticRegression(solver='lbfgs', random_state=1)
SM_model.fit(SM_X_resampled, SM_y_resampled)

LogisticRegression(random_state=1)

In [56]:
# Display the confusion matrix
SM_y_pred = SM_model.predict(SM_X_resampled)
confusion_matrix(SM_y_resampled, SM_y_pred)

array([[55419,    94],
       [    0, 55928]])

In [57]:
# Print the imbalanced classification report
print(classification_report_imbalanced(SM_y_resampled, SM_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      1.00      1.00      1.00      1.00      1.00     55513
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     55928

avg / total       1.00      1.00      1.00      1.00      1.00      1.00    111441



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
